In [ ]:
# Useful Links
# 1. https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# 2. https://machinelearningknowledge.ai/keras-implementation-of-resnet-50-architecture-from-scratch/
# 3. https://towardsdatascience.com/resnets-for-cifar-10-e63e900524e0 (Understanding the resnet architecture for cifar10)


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
PATH = '/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip4/'

Mounted at /content/drive/


In [ ]:
# Load necessary libraries
for i in range(5):
  import os
  import pandas as pd
  import numpy as np
  #import cv2
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import backend as K
  from tensorflow.keras.optimizers import SGD, Adam
  #from google.colab.patches import cv2_imshow
  from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau, CSVLogger
  from tensorflow.keras.models import Sequential, Model,load_model
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
  from tensorflow.keras.preprocessing import image
  from tensorflow.keras.initializers import glorot_uniform
  from matplotlib import pyplot as plt
  from tensorflow.keras.utils import to_categorical, plot_model



  def load_dataset(dName="CIFAR10"):
      dataset = None
      num_classes = None
      if dName == "CIFAR10":
          num_classes = 10
          dataset = tf.keras.datasets.cifar10.load_data()
      if dName == "CIFAR100":
          num_classes = 100
          dataset = tf.keras.datasets.cifar100.load_data()
      (X_train, y_train), (X_test, y_test) = dataset
      # Convert target value to categorical values
      # One-hot-encoded target values
      y_train = to_categorical(y_train,num_classes)
      y_test = to_categorical(y_test, num_classes)
      
      return (X_train, y_train),(X_test, y_test)
  def divideDataset(X_train, y_train, X_test,y_test):
      dataLength = X_train.shape[0]
      trainLen=0
      dataTrain = []
      dataTest = []
      percent = 0.2
      while(trainLen<dataLength):
          trainLen = int(X_train.shape[0]*percent)
          #print(X_train[:trainLen].shape)
          train = (X_train[:trainLen],y_train[:trainLen])
          retriveLen = int(X_test.shape[0]*percent)
          test = (X_test[:retriveLen],y_test[:retriveLen])
          #print(tuple(train))
          dataTrain.append(train)
          dataTest.append(test)
          #print("\n")
          percent +=0.2
      return dataTrain, dataTest
  (X_train, y_train),(X_test, y_test) = load_dataset()
  dataTrain, dataTest = divideDataset(X_train, y_train, X_test, y_test) # this contains the list of 5 different datasets
  def normalizeInput(X_train,X_test):
      X_train = X_train.astype('float32')
      X_test = X_test.astype('float32')
      mean = np.mean(X_train, axis=(0,1,2,3))
      std = np.std(X_train,axis=(0,1,2,3))
      X_train = (X_train-mean)/(std+1e-7)
      X_test = (X_test-mean)/(std+1e-7)
      
      return X_train, X_test
  X_train, X_test = normalizeInput(dataTrain[0][0], dataTest[0][0])
  y_train = dataTrain[0][1]
  y_test = dataTest[0][1]
  # RESNET50 WITH SKIP CONNECTION 4: Implemented by Lokesh
  def createStageOne(x, f,k, s,p,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    block_name = 'stage_1'+counter
    x_skip = x
    layers = len(f)
    for i in range(layers):
      x = Conv2D(f[i],kernel_size=(k[i],k[i]),strides=(s[i],s[i]),padding=p[i])(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
    x = Conv2D(f[layers-1],kernel_size=(k[layers-1],k[layers-1]),strides=(s[layers-1],s[layers-1]),padding=p[layers-1],name=block_name+'b')(x)
    x = BatchNormalization()(x)
    print(x.shape)

    x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 1:", (x_skip.shape,x.shape))

    return x

  def createStageTwo_(x, f,k, s,p,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    block_name = 'stage_2'+counter
    x_skip = x
    #print("Recevied Input at Stage 2: ", x.shape)
    layers = len(f)
    for i in range(layers):
      x = Conv2D(f[i],kernel_size=(k[i],k[i]),strides=(s[i],s[i]),padding=p[i])(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
    x = Conv2D(f[layers-1],kernel_size=(k[layers-1],k[layers-1]),strides=(s[layers-1],s[layers-1]),padding=p[layers-1],name=block_name+'b')(x)
    x = BatchNormalization()(x)
    print(x.shape)

    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)

    x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 2:", (x_skip.shape,x.shape))

    return x

  def createStageThree_(x, f,k, s,p,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    block_name = 'stage_3'+counter
    x_skip = x
    #print("Recevied Input at Stage 3: ", x.shape)
    layers = len(f)
    for i in range(layers):
      x = Conv2D(f[i],kernel_size=(k[i],k[i]),strides=(s[i],s[i]),padding=p[i])(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
    x = Conv2D(f[layers-1],kernel_size=(k[layers-1],k[layers-1]),strides=(s[layers-1],s[layers-1]),padding=p[layers-1],name=block_name+'b')(x)
    x = BatchNormalization()(x)
    #print(x.shape)

    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)

    x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 3:", (x_skip.shape,x.shape))

    return x
  def createStageFour_(x, f,k, s,p,counter):
    '''
    Argument
    x - input
    f - filter_size
    k - kernel size
    s - stride
    p - padding
    '''
    block_name = 'stage_4'+counter
    x_skip = x
    #print("Recevied Input at Stage 4: ", x.shape)
    layers = len(f)
    for i in range(layers):
      x = Conv2D(f[i],kernel_size=(k[i],k[i]),strides=(s[i],s[i]),padding=p[i])(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
    x = Conv2D(f[layers-1],kernel_size=(k[layers-1],k[layers-1]),strides=(s[layers-1],s[layers-1]),padding=p[layers-1],name=block_name+'b')(x)
    x = BatchNormalization()(x)
    #print(x.shape)

    if x.shape[1] != x_skip.shape[1]:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(2,2),padding='valid')(x_skip)
    else:
      x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)

    x_skip = Conv2D(f[layers-1],kernel_size=(1,1),strides=(1,1),padding='valid')(x_skip)
    x_skip = BatchNormalization()(x_skip)
    x = Add()([x,x_skip])
    x = Activation('relu')(x)
    #print("Stage 4:", (x_skip.shape,x.shape))

    return x
  def resnet50():
    inputs = Input(shape = (32,32,3), name = "image_input")
    x = ZeroPadding2D((3,3))(inputs)
    # Initial Convolutional block
    
    x = Conv2D(64, kernel_size=(7,7), strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #print("before Max pooling: ", x.shape)
    x = MaxPooling2D((3,3),strides=(2,2))(x)
    x = createStageOne(x,[64,64,256,64],[1,3,1,1],[1,1,1,1],['valid','same','valid','valid'],'1')
    x = createStageOne(x,[64,256,64,64],[3,1,1,3],[1,1,1,1],['same','valid','valid','same'],'2')

    x = createStageTwo_(x,[256,128,128,512],[1,1,3,1],[1,2,1,1],['valid','valid','same','valid'],'3')
    x = createStageTwo_(x,[128,128,512,128],[1,3,1,1],[1,1,1,1],['valid','same','valid','valid'],'4')
    x = createStageTwo_(x,[128,512,128,128],[3,1,1,3],[1,1,1,1],['same','valid','valid','same'],'5')

    x = createStageThree_(x, [512,256,256,1024],[1,1,3,1],[1,2,1,1],['valid','valid','same','valid'],'6')
    x = createStageThree_(x, [256,256,1024,256],[1,3,1,1],[1,1,1,1],['valid','same','valid','valid'],'7')
    x = createStageThree_(x, [256,1024,256,256],[3,1,1,3],[1,1,1,1],['same','valid','valid','same'],'8')
    x = createStageThree_(x, [1024,256,256,1024],[1,1,3,1],[1,1,1,1],['valid','valid','same','valid'],'9')


    x = createStageFour_(x,[256,256,1024,512],[1,3,1,1],[1,1,1,2],['valid','same','valid','valid'],'10')
    x = createStageFour_(x,[512,2048,512,512],[3,1,1,3],[1,1,1,1],['same','valid','valid','same'],'11')
    x = createStageFour_(x,[2048,512,512,2048],[1,1,3,1],[1,1,1,1],['valid','valid','same','valid'],'12')

    x = AveragePooling2D((2,2),padding='same')(x)
    x = Flatten()(x)
    x = Dense(10,activation='softmax')(x)
    model = Model(inputs=inputs,outputs=x, name='LResNet50')
    #print(x.shape)
    return model

  model = resnet50()
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  #model.summary()
  X_train, X_test = normalizeInput(dataTrain[i][0], dataTest[i][0])
  y_train = dataTrain[i][1]
  y_test = dataTest[i][1]
  print("Len: ",len(X_train))
  csvLogger = CSVLogger('/content/drive/MyDrive/PhD Studies/fall21/ADVML/results/skip4/'+str(i)+'.csv')
  earlyStop = EarlyStopping(monitor='loss',patience=5,mode='auto')
  history = model.fit(X_train,y_train, epochs=300,batch_size=1024,validation_data=(X_test,y_test),verbose=1,callbacks=[csvLogger,earlyStop])

(None, 7, 7, 64)
(None, 7, 7, 64)
(None, 4, 4, 512)
(None, 4, 4, 128)
(None, 4, 4, 128)
Len:  10000
Epoch 1/300
10/10 [==============================] - 11s 387ms/step - loss: 3.3180 - accuracy: 0.1160 - val_loss: 2.3086 - val_accuracy: 0.1105
Epoch 2/300
10/10 [==============================] - 3s 260ms/step - loss: 2.3632 - accuracy: 0.1660 - val_loss: 2.3142 - val_accuracy: 0.0990
Epoch 3/300
10/10 [==============================] - 3s 259ms/step - loss: 2.1372 - accuracy: 0.2105 - val_loss: 2.3341 - val_accuracy: 0.0990
Epoch 4/300
10/10 [==============================] - 3s 260ms/step - loss: 2.0069 - accuracy: 0.2446 - val_loss: 2.3912 - val_accuracy: 0.0990
Epoch 5/300
10/10 [==============================] - 3s 260ms/step - loss: 1.9015 - accuracy: 0.2873 - val_loss: 2.4645 - val_accuracy: 0.0990
Epoch 6/300
10/10 [==============================] - 3s 259ms/step - loss: 1.7413 - accuracy: 0.3547 - val_loss: 2.5796 - val_accuracy: 0.0990
Epoch 7/300
10/10 [======================